## Adjacency list construction -- example : EU NUTS2 regions
shp download : https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts <br>
original idea developed by Eszter Bokányi (bokae on github)

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# NUTS-3 level shapefile
map_df = gpd.read_file('../data/shapefile/NUTS_RG_10M_2016_3035.shp')

# FILTER for NUTS-2
map_df = map_df[map_df['LEVL_CODE']<3]

# keep 4-digit region codes
map_df['digits'] = map_df['NUTS_ID'].apply(lambda x: len(x))
map_df = map_df[map_df['digits'] == 4]

In [3]:
# create a spatial index for faster join
map_df["buffered_geometry"] = map_df.buffer(250)
si = map_df.set_geometry("buffered_geometry").sindex

In [4]:
# possible matches based on spatial index
map_df["possible_matches"] = map_df.buffer(250).map(lambda p: list(si.intersection(p.bounds)))

# exact matches calculated only for possible matches, excluding the polygon in question
map_df["neighbors"] = map_df.apply(lambda row: [map_df.index[e] for e in row["possible_matches"] if row["buffered_geometry"].intersects(map_df["buffered_geometry"].iloc[e]) and map_df.index[e]!=row.name], axis=1)

In [5]:
# construct the final dataframe
adj_df = map_df.loc[:,['NUTS_ID', 'neighbors']].dropna()
adj_df = adj_df[adj_df['neighbors'].map(len) >0]
adj_df = adj_df.explode('neighbors')
adj_df['neighbors'] = adj_df['neighbors'].map(map_df['NUTS_ID'].to_dict())

In [6]:
# save the final adjacency list
adj_df.to_csv('../data/adj_list_NUTS2.csv', sep=';', index=False)